In [2]:
!pip install google-cloud-storage


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=VU3NoyTgc24d3ZI475Ar27CojbFkMC&access_type=offline&code_challenge=pjjOaxUFRQfW1bY2f69_9dJwpinS23a1qW1hNnzo628&code_challenge_method=S256


Credentials saved to file: [/Users/thomasvrancken/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "example-project-449313" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [4]:
!gcloud auth list

   Credentialed Accounts
ACTIVE  ACCOUNT
*       t.vrancken@uliege.be

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [5]:
from google.cloud import storage

# Define variables
bucket_name = "example-bucket-mlsd"  # Replace with your bucket name
source_file_path = "data/housing.csv"  # Path to your local CSV file
destination_blob_name = "other_folder/destination_file.csv"  # Path in GCS

def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to Google Cloud Storage."""
    # Initialize GCS client (it will use default credentials)
    client = storage.Client()

    # Get bucket
    bucket = client.bucket(bucket_name)

    # Create a blob (GCS object) and upload the file
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)

    print(f"File {source_file_path} uploaded to {bucket_name}/{destination_blob_name}")

# Call the function
upload_to_gcs(bucket_name, source_file_path, destination_blob_name)

File data/housing.csv uploaded to example-bucket-mlsd/other_folder/destination_file.csv


In [6]:
def download_from_gcs(bucket_name, source_blob_name, destination_file_path):
    """Downloads a file from Google Cloud Storage."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_path)

    print(f"File {source_blob_name} downloaded to {destination_file_path}")

# Define variables
bucket_name = "example-bucket-mlsd"  # Replace with your bucket name
source_blob_name = "other_folder/destination_file.csv"  # Path in GCS
destination_file_path = "data/downloaded_file.csv"  # Local path to save the file

# Call the function
download_from_gcs(bucket_name, source_blob_name, destination_file_path)

File other_folder/destination_file.csv downloaded to data/downloaded_file.csv


In [8]:
!export GOOGLE_APPLICATION_CREDENTIALS="/Users/thomasvrancken/Documents/project/ulg/github/info9023-mlops-main/demos/04_gcs_bq/secrets/example-project-449313-da2a19439135.json"

In [9]:
client = storage.Client.from_service_account_json('/Users/thomasvrancken/Documents/project/ulg/github/info9023-mlops-main/demos/04_gcs_bq/secrets/example-project-449313-da2a19439135.json')

In [10]:
!pip install google-cloud-bigquery pandas


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
from google.cloud import bigquery
import pandas as pd

# Initialize BigQuery client
client = bigquery.Client()

table_path = f"example-project-449313.example_dataset_mlsd.example_table_housing_2"

# Define SQL query (Example: Check distinct values for "ocean_proximity")
query = f"""
SELECT DISTINCT(ocean_proximity) FROM `{table_path}`
"""

# Execute query and convert result to a DataFrame
query_job = client.query(query)
df = query_job.to_dataframe()

# Display result
print(df)

/Users/thomasvrancken/Documents/project/ulg/github/info9023-mlops-main/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  ocean_proximity
0      NEAR OCEAN
1       <1H OCEAN
2          INLAND
3          ISLAND
4        NEAR BAY


In [15]:
# Define SQL query 
query = f"""
SELECT AVG(median_house_value) AS avg_median_house_value
FROM `{table_path}`
WHERE ocean_proximity="INLAND"
"""

# Execute query and convert result to a DataFrame
query_job = client.query(query)
df = query_job.to_dataframe()

# Display result
print(df)

/Users/thomasvrancken/Documents/project/ulg/github/info9023-mlops-main/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   avg_median_house_value
0           124805.392001
